In [9]:
import pandas as pd
import nltk

In [2]:
data = pd.read_csv("stockprice_15_04_2019-25_10_2017.csv")
nabil_data = data.loc[data['traded_companies'] == "Nabil Bank Limited"]
nabil_data["date"] = pd.to_datetime(nabil_data["date"])
nabil_data.set_index("date", inplace = True)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
nabil_data.tail()

,traded_companies,no_of_transaction,max_price,min_price,closing_price,traded_shares,amount,previous_closing,difference_rs
date,,,,,,,,,
2017-10-18,Nabil Bank Limited,42.0,1280.0,1264.0,1277.0,4812.0,6117000.0,1273.0,4.0
2017-10-22,Nabil Bank Limited,16.0,1285.0,1270.0,1272.0,1070.0,1368105.0,1277.0,-5.0
2017-10-23,Nabil Bank Limited,30.0,1280.0,1260.0,1265.0,1708.0,2169615.0,1272.0,-7.0
2017-10-24,Nabil Bank Limited,24.0,1270.0,1251.0,1257.0,2908.0,3659420.0,1265.0,-8.0
2017-10-25,Nabil Bank Limited,32.0,1265.0,1235.0,1235.0,2350.0,2930820.0,1257.0,-22.0


In [17]:
news_data = pd.read_csv("processed_data.csv")
news_data.tail()

,Unnamed: 0,date,news
39584,39584,2011-03-28,"['gurkha_development_bank', 'limited', 'has', ..."
39585,39585,2011-03-28,"['27', 'march,2011', ':', 'kaski', 'finance', ..."
39586,39586,2011-03-27,"['citizens_bank', 'interational', 'ltd.', 'is'..."
39587,39587,2011-03-27,"['income', 'from', 'tourism', 'declines', ':',..."
39588,39588,2011-03-27,"['lumbini', 'bank_ltd.', 'has', 'shifted', 'it..."


In [18]:
news = news_data.groupby("date")['news'].apply(list)


In [19]:
news = pd.DataFrame(news)

In [20]:
result_df = nabil_data.join(news, how = "inner")

In [24]:
result_df.head()

,traded_companies,no_of_transaction,max_price,min_price,closing_price,traded_shares,amount,previous_closing,difference_rs,news
date,,,,,,,,,,
2011-03-27,Nabil Bank Limited,14.0,1160.0,1120.0,1147.0,1426.0,1619270.0,1174.0,-27.0,"[['citizens_bank', 'interational', 'ltd.', 'is..."
2011-03-28,Nabil Bank Limited,12.0,1150.0,1110.0,1132.0,352.0,393534.0,1147.0,-15.0,"[['capital', 'merchant', 'bank', '&', 'finance..."
2011-03-29,Nabil Bank Limited,24.0,1120.0,1050.0,1097.0,1456.0,1576420.0,1132.0,-35.0,"[['bhargav', 'bikash', 'bank_ltd.', 'has', 'pu..."
2011-03-30,Nabil Bank Limited,16.0,1076.0,1030.0,1040.0,1340.0,1394926.0,1097.0,-57.0,"[['finance', 'secretary', 'khanal', 'resigns',..."
2011-03-31,Nabil Bank Limited,23.0,1109.0,1020.0,1082.0,1828.0,1947570.0,1040.0,42.0,"[['real', 'sector', 'fails', 'to', 'make', 'im..."


In [25]:
import gensim

In [29]:
model = gensim.models.KeyedVectors.load("model")

In [32]:
model.get_vector("nabil")

array([ 1.3258145 , -0.76911294,  0.99010336, -1.52899   , -1.0176576 ,
        1.348651  , -0.25274074,  1.582532  ,  0.14764838,  0.94472724,
       -0.37010437, -1.4983261 , -0.5293481 , -1.5408803 ,  0.21962017,
        0.57169455,  0.43515724,  0.03204608,  0.5487077 , -1.5833049 ,
        0.7968877 , -0.5915963 , -0.62255037, -0.47957012,  0.90518034,
        0.5615598 , -0.58158714,  0.4645961 , -1.4251713 ,  0.08732081,
        0.84663564,  0.7135513 ,  1.1959249 ,  0.47754547,  0.01434105,
       -1.6176889 , -0.586676  ,  0.10666194, -1.119466  , -0.9800852 ,
       -1.2426282 ,  0.27884215,  0.8489607 ,  1.6138057 ,  0.41307768,
       -0.5791473 , -0.40996104,  0.3306974 , -1.2260319 , -0.2630171 ,
       -0.11440781,  0.44768652,  1.3737814 ,  1.4242519 , -0.7581244 ,
        0.24446177, -0.19850731,  0.23978609, -0.57753336, -0.8013171 ,
        2.089949  ,  1.2672119 ,  1.8016299 , -0.5623023 , -1.1486661 ,
       -0.07659995, -0.16400583, -0.6348515 , -0.84501326,  0.03

In [40]:
import numpy as np
def vectorize(news):
    result = []
    for new in news:
        li = list(map(model.get_vector, new))
        result.append( np.average(li, axis = 0))
    return result
    
        

In [43]:
result_df.news = result_df.news.apply(vectorize)

In [46]:
np.array(result_df.head(1).news)

array([list([array([ 2.75943220e-01,  6.26355588e-01,  6.06337905e-01, -5.18821061e-01,
        6.88847974e-02, -8.70770276e-01,  7.30903327e-01, -2.80250788e-01,
        1.50923952e-01,  4.26712930e-02, -8.50091457e-01, -6.48530796e-02,
       -2.73776740e-01,  6.96022749e-01,  2.09348163e-05, -3.44716460e-01,
        1.04149587e-01, -6.46334887e-01, -1.24564394e-01, -9.40130353e-01,
       -1.74250916e-01,  2.58777887e-01,  1.22607119e-01, -3.45943153e-01,
       -1.63669646e-01, -5.18338859e-01, -4.13281977e-01,  5.19943714e-01,
       -2.69818276e-01,  8.03209364e-01,  4.73370939e-01,  4.89581436e-01,
        4.79189247e-01, -4.23913859e-02,  1.87810168e-01, -7.74231777e-02,
       -2.17911992e-02,  3.18085402e-02, -1.91615567e-01, -1.75861523e-01,
       -7.59915054e-01, -6.30263314e-02,  4.35225777e-02,  1.17870080e+00,
        6.47207260e-01,  8.08237314e-01, -3.55857521e-01, -1.04694389e-01,
        2.96265334e-01, -6.91980645e-02, -1.98170751e-01,  2.48677313e-01,
       -5.22

In [50]:
result_df[["closing_price", "news"]].head()

,closing_price,news
date,,
2011-03-27,1147.0,"[[0.27594322, 0.6263556, 0.6063379, -0.5188210..."
2011-03-28,1132.0,"[[0.21920921, 0.7366271, 0.6421762, -0.5315173..."
2011-03-29,1097.0,"[[0.17657533, 0.77826136, 0.63073933, -0.50873..."
2011-03-30,1040.0,"[[0.24216579, 0.6578429, 0.73102975, -0.568503..."
2011-03-31,1082.0,"[[0.24721189, 0.8349754, 0.68579185, -0.577519..."
